# Hardware Crypto Attack


So far we have mostly been talking about software crypto. But how can we expand this to hardware crypto? Luckily it takes very few changes, so you don't have much to do!

In this lab we'll be looking at what is required to attack a hardware crypto device, and what sort of attacks work on these devices. In this case we're going to "cheat" and use an already recorded power trace, since we don't have hardware crypto on our target boards.

## Capture

In [ ]:
%%bash
cd ../hardware/victims/firmware/
mkdir -p simpleserial-aes-lab1 && cp -r simpleserial-aes/* $_

In [ ]:
%%bash
cd ../hardware/victims/firmware/simpleserial-aes-lab1
make PLATFORM=CW308_STM32F4 CRYPTO_TARGET=HWAES

In [ ]:
import chipwhisperer as cw
scope = cw.scope()
target = cw.target(scope)

In [ ]:
%run "Helper_Scripts/Setup_Target_Generic.ipynb"


scope.adc.samples = 2000
scope.gain.mode = "low"
scope

In [ ]:
prog = cw.programmers.STM32FProgrammer
fw_path = "../hardware/victims/firmware/simpleserial-aes-lab1/simpleserial-aes-CW308_STM32F4.hex"
cw.programTarget(scope, prog, fw_path)

In [ ]:
project = cw.createProject("stm32f415_lab.cwp", overwrite=True)
tc = project.newSegment()

#Capture Traces
from tqdm import tqdm
import numpy as np
import time

ktp = cw.ktp.Basic(target=target)

N = 5000  # Number of traces
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here

    target.loadEncryptionKey(key)
    target.loadInput(text)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    
    # note you may want:
    # num_char = target.ser.inWaiting()
    # response = target.ser.read(num_char)
    textout = target.readOutput()  # clears the response from the serial port
    #traces.append(scope.getLastTrace())
    tc.addTrace(scope.getLastTrace(), text, textout, key)
    
project.appendSegment(tc)
project.save()

# cleanup the connection to the target and scope
scope.dis()
target.dis()

## Analysis

Next, we'll add our traces to a preprocessing module. We can feed `project.traceManager()` right into `attack.setTraceSource()`, but we could also add pre-processing inbetween (more about this later). We'll also re-open the traces, in this case it is required since the call to `closeAll()` would have flushed the buffers.

In [ ]:
#Force reload of project data (if you comment out 'closeAll()' this isn't needed)

#We also rebuild the project object in case you only want to run this half
project = cw.openProject('./stm32f415_lab.cwp')

tm = project.traceManager()

This time we're going to do a few things. First we will get the traces, and plot a few of them as-is. You can adjust the traces plotted by adjusting the `range(10)`. For example `range(1)` plots the first trace.

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Dark2_5 as palette
import itertools  

output_notebook()
p = figure(sizing_mode='scale_width', plot_height=300)

# create a color iterator
colors = itertools.cycle(palette)  

x_range = range(0, tm.numPoints())
for i, color in zip(range(10), colors): #Adjust range(n) to plot certain traces
    p.line(x_range, tm.getTrace(i), color=color)
show(p)

If this all works - let's just continue the attack! We're going to use the same leakge model as previously (Hamming weight), we'll seperate this out since will be changing that model around shortly.

In [ ]:
leak_model = cw.AES128(cw.AES128Leakage.SBox_output)

In [ ]:
attack = cw.CPA(project.traceManager(), leak_model)
attack.setReportingInterval(50)
attack.setPointRange((1312, 1317))

And then actually run it:

In [ ]:
cb = cw.getJupyterCallback(attack)
attack_results = attack.processTracesNoGUI(cb, show_progress_bar=True)

This will almost certainly fail. The leakage model is incorrect, so we need to find the correct (new) leakage model we should be using. This turns out to be pretty easy, since most typical hardware implementations use only one of a few possible models. We'll specifically just try the "Last Round State Over-Written" model first. You can do this by updating the model above to have the following:

In [ ]:
leak_model = cw.AES128(cw.AES128Leakage.LastroundStateDiff)

But this might not be enough! You may need to "window" around the area of interest. This is best done by plotting the results, and picking a nice area. For example a window of `attack.setPointRange((1312, 1314))` seems to work well on some traces.

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()

bnum = 11

key = attack.knownKey()
data = attack.getStatistics().diffs[bnum]
xr = range(0, len(data[0]))

for v in range(0, 256):
    p.line(xr, data[v], line_color='green')

p.line(xr, data[key[bnum]], line_color='red')
show(p)

You should see a graph of red and green in time (samples). In red is the correlation of the correct subkey for the first byte, while the rest are in green. You can use this graph to help fine-tune the windowing of the data.

## Conclusion

Attacking hardware crypto is similar to any other DPA style attack. In this example we have concentrated on the standard "Last Round State" to break a real hardware accelerator.